In [1]:
import os
import json
from collections import Counter

# --- 配置区 ---

# 1. 定义输入文件与它们缩写的映射关系
#    脚本会根据这个字典来寻找文件并生成新的字段
FILE_MAPPING = {
    "FirstAgentDataHighLevel_multiple_10.json": "fh",
    "FirstAgentDataLowLevel_multiple_10.json": "fl",
    "ThirdAgentDataHighLevel_multiple_10.json": "th",
    "ThirdAgentDataLowLevel_multiple_10.json": "tl",
}

# 2. 定义输入文件所在的目录（"." 表示当前目录）
INPUT_DIR = "/Users/jinyi/Documents/code/memory/mem0/evaluation/dataset/Membench/data2test" 

# 3. 定义合并后输出的文件名和目录
OUTPUT_DIR = "processed_data"
OUTPUT_FILENAME = "merged_test_data.json"

# --- 脚本主逻辑 ---

def merge_and_analyze_data():
    """
    主函数，用于合并、处理JSON文件并进行统计分析。
    """
    merged_data = []
    category_counter = Counter()

    print("--- 开始处理JSON文件 ---")

    # 遍历在配置中定义的每一个文件
    for filename, short_name in FILE_MAPPING.items():
        file_path = os.path.join(INPUT_DIR, filename)

        # 检查文件是否存在
        if not os.path.exists(file_path):
            print(f"⚠️ 警告：文件 '{filename}' 不存在，已跳过。")
            continue

        print(f"正在读取和处理: {filename}...")

        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # 根据你提供的格式，数据嵌套在两层动态的键下
        # 例如：{"Emotion": {"Emotion": [...]}}
        for top_category, inner_dict in data.items():
            for inner_category, records_list in inner_dict.items():
                
                # 遍历核心记录列表
                for record in records_list:
                    # 获取tid，如果不存在则给一个默认值
                    tid = record.get("tid", "UNKNOWN_TID")

                    # 1. 创建新的字段
                    classification = f"{short_name}_{top_category}_{inner_category}"
                    test_id = f"{classification}_{tid}"

                    # 2. 为当前记录添加5个新字段
                    record['test_id'] = test_id
                    record['classification'] = classification
                    record['file_type'] = short_name
                    record['top_category'] = top_category
                    record['inner_category'] = inner_category
                    
                    # 3. 将处理后的记录添加到总列表中
                    merged_data.append(record)
                    
                    # 4. 更新分类计数器
                    category_counter[classification] += 1
    
    print("\n--- 所有文件处理完毕 ---")

    # 确保输出目录存在
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)
        print(f"已创建输出目录: {OUTPUT_DIR}")

    # 将合并后的数据写入新文件
    output_path = os.path.join(OUTPUT_DIR, OUTPUT_FILENAME)
    with open(output_path, 'w', encoding='utf-8') as f:
        # 使用 indent=4 格式化JSON，方便阅读
        json.dump(merged_data, f, indent=4, ensure_ascii=False)
    
    print(f"\n✅ 合并后的文件已成功保存至: {output_path}")

    # 打印统计指标
    print("\n--- 数据统计指标 ---")
    if not category_counter:
        print("未处理任何数据。")
        return
        
    print("每个分类下的题目数量:")
    # 为了美观，找到最长的分类名以对齐输出
    max_len = max(len(cat) for cat in category_counter.keys())
    for category, count in sorted(category_counter.items()):
        print(f"  - {category:<{max_len}} : {count} 条")
        
    total_questions = sum(category_counter.values())
    print("-" * (max_len + 10))
    print(f"  - {'总计':<{max_len}} : {total_questions} 条")
    print("\n--- 任务完成 ---")


# 当脚本直接运行时，执行主函数
if __name__ == "__main__":
    merge_and_analyze_data()

--- 开始处理JSON文件 ---
正在读取和处理: FirstAgentDataHighLevel_multiple_10.json...
正在读取和处理: FirstAgentDataLowLevel_multiple_10.json...
正在读取和处理: ThirdAgentDataHighLevel_multiple_10.json...
正在读取和处理: ThirdAgentDataLowLevel_multiple_10.json...

--- 所有文件处理完毕 ---
已创建输出目录: processed_data

✅ 合并后的文件已成功保存至: processed_data/merged_test_data.json

--- 数据统计指标 ---
每个分类下的题目数量:
  - fh_Emotion_Emotion                     : 30 条
  - fh_Preference_book                     : 30 条
  - fh_Preference_food                     : 30 条
  - fh_Preference_movie                    : 30 条
  - fl_Aggregative_events                  : 23 条
  - fl_Aggregative_roles                   : 23 条
  - fl_Comparative_events                  : 23 条
  - fl_Comparative_roles                   : 23 条
  - fl_Knowledge_updating_events           : 23 条
  - fl_Knowledge_updating_roles            : 23 条
  - fl_Multi-hop_events                    : 23 条
  - fl_Multi-hop_roles                     : 23 条
  - fl_Multi-session-assistant_multi_agent : 23

In [3]:
import os
import json
from collections import Counter
# 3. 定义合并后输出的文件名和目录
OUTPUT_DIR = "processed_data"
OUTPUT_FILENAME = "merged_test_data.json"

# 读取合并后的数据文件
output_path = os.path.join(OUTPUT_DIR, OUTPUT_FILENAME)
with open(output_path, 'r', encoding='utf-8') as f:
    merged_data = json.load(f)

# 按照 classification 统计各类别题目数量

category_counter = Counter()
for record in merged_data:
    classification = record.get('inner_category', 'UNKNOWN')
    category_counter[classification] += 1

# 打印统计结果
max_len = max(len(cat) for cat in category_counter.keys())
print("每个分类下的题目数量:")
for category, count in sorted(category_counter.items()):
    print(f"  - {category:<{max_len}} : {count} 条")
print("-" * (max_len + 10))
print(f"  - {'总计':<{max_len}} : {sum(category_counter.values())} 条")

每个分类下的题目数量:
  - Emotion     : 45 条
  - book        : 68 条
  - events      : 210 条
  - food        : 68 条
  - hybrid      : 60 条
  - items       : 36 条
  - movie       : 68 条
  - multi_agent : 23 条
  - places      : 36 条
  - roles       : 210 条
---------------------
  - 总计          : 824 条
